In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - PaliGemma (Deployment)

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_jax_paligemma_deployment.ipynb">
      <img alt="Workbench logo" src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" width="32px"><br> Run in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_jax_paligemma_deployment.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_jax_paligemma_deployment.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates deploying PaliGemma to a Vertex AI Endpoint and making online predictions for tasks listed below. The notebook also demonstrates creating a shareable link to a web interface that allows querying with the deployed PaliGemma model using [Gradio](https://www.gradio.app/).


### Objective

- Deploy PaliGemma to a Vertex AI Endpoint.
- Make predictions to the endpoint including:
  - Answering questions about a given image.
  - Captioning images.
  - Extracting texts.
  - Detecting objects.
- Create a playground website to use with the PaliGemma Vertex AI Endpoint.

### File a bug

File a bug on [GitHub](https://github.com/GoogleCloudPlatform/vertex-ai-samples/issues/new) if you encounter any issue with the notebook.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

In [ ]:
# @title Setup Google Cloud project
# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

# @markdown 3. If you want to run predictions with A100 80GB or H100 GPUs, we recommend using the regions listed below. **NOTE:** Make sure you have associated quota in selected regions. Click the links to see your current quota for each GPU type: [Nvidia A100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_a100_80gb_gpus), [Nvidia H100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_h100_gpus). You can request for quota following the instructions at ["Request a higher quota"](https://cloud.google.com/docs/quota/view-manage#requesting_higher_quota).

# @markdown | Machine Type | Accelerator Type | Recommended Regions |
# @markdown | ----------- | ----------- | ----------- |
# @markdown | a2-ultragpu-1g | 1 NVIDIA_A100_80GB | us-central1, us-east4, europe-west4, asia-southeast1, us-east4 |
# @markdown | a3-highgpu-2g | 2 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-4g | 4 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-8g | 8 NVIDIA_H100_80GB | us-central1, europe-west4, us-west1, asia-southeast1 |

# Upgrade Vertex AI SDK.
! pip3 install --upgrade --quiet 'google-cloud-aiplatform==1.103.0'

# Import the necessary packages
! pip install -q gradio==4.21.0
import enum
import importlib
import io
import os
import re
from typing import Sequence, Tuple

import gradio as gr
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from google.cloud import aiplatform
from PIL import Image

# Upgrade Vertex AI SDK.
if os.environ.get("VERTEX_PRODUCT") != "COLAB_ENTERPRISE":
    ! pip install --upgrade tensorflow
! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)

LABEL = "endpoint"
models, endpoints = {}, {}


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION)

! gcloud config set project $PROJECT_ID

import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=REGION,
)

# @markdown ### Access PaliGemma models on Vertex AI for GPU based serving
# @markdown Accept the model agreement to access the models:
# @markdown 1. Open the [PaliGemma model card](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/363) from [Vertex AI Model Garden](https://cloud.google.com/model-garden).
# @markdown 1. Review and accept the agreement in the pop-up window on the model card page. If you have previously accepted the model agreement, there will not be a pop-up window on the model card page and this step is not needed.
# @markdown 1. After accepting the agreement of PaliGemma, a `gs://` URI containing PaliGemma pretrained models will be shared.
# @markdown 1. Paste the link in the `VERTEX_AI_MODEL_GARDEN_PALIGEMMA` field below.
# @markdown 1. The PaliGemma models will be copied into `BUCKET_URI`.
# @markdown The file transfer can take anywhere from 15 minutes to 30 minutes.
VERTEX_AI_MODEL_GARDEN_PALIGEMMA = "gs://"  # @param {type:"string", isTemplate:true}
assert (
    VERTEX_AI_MODEL_GARDEN_PALIGEMMA
), "Click the agreement of PaliGemma in Vertex AI Model Garden, and get the GCS path of PaliGemma model artifacts."

## Deploy PaliGemma to a Vertex AI Endpoint

In [ ]:
# @title Select the model variants

pretrained_filename_lookup = {
    "paligemma-224-float32": "pt_224.npz",
    "paligemma-224-float16": "pt_224.f16.npz",
    "paligemma-224-bfloat16": "pt_224.bf16.npz",
    "paligemma-448-float32": "pt_448.npz",
    "paligemma-448-float16": "pt_448.f16.npz",
    "paligemma-448-bfloat16": "pt_448.bf16.npz",
    "paligemma-896-float32": "pt_896.npz",
    "paligemma-896-float16": "pt_896.f16.npz",
    "paligemma-896-bfloat16": "pt_896.bf16.npz",
    "paligemma-mix-224-float32": "mix_224.npz",
    "paligemma-mix-224-float16": "mix_224.f16.npz",
    "paligemma-mix-224-bfloat16": "mix_224.bf16.npz",
    "paligemma-mix-448-float32": "mix_448.npz",
    "paligemma-mix-448-float16": "mix_448.f16.npz",
    "paligemma-mix-448-bfloat16": "mix_448.bf16.npz",
}

# @markdown Select the desired resolution and precision of prebuilt model to deploy, leaving the optional `custom_paligemma_model_uri` as is. Higher resolution and precision_type can result in better inference results, but may require additional GPU.

# @markdown You can also serve a finetuned PaliGemma model by setting `resolution` and `precision_type` to the resolution and precision type of the original base model and then setting `custom_paligemma_model_uri` to the GCS URI containing the model.

# @markdown **Note**: You cannot use accelerator type `NVIDIA_TESLA_V100` to serve prebuilt or finetuned PaliGemma models with resolution `896` and precision_type `float32`.

model_variant = "mix"  # @param ["mix", "pt"]
resolution = 224  # @param [224, 448, 896]
precision_type = "float32"  # @param ["float32", "float16", "bfloat16"]
custom_paligemma_model_uri = "gs://"  # @param {type: "string"}

if model_variant == "mix":
    model_name_prefix = "paligemma-mix"
else:
    model_name_prefix = "paligemma"


if custom_paligemma_model_uri == "gs://" or not custom_paligemma_model_uri:
    model_name = f"{model_name_prefix}-{resolution}-{precision_type}"
    checkpoint_filename = pretrained_filename_lookup[model_name]
    checkpoint_path = os.path.join(
        VERTEX_AI_MODEL_GARDEN_PALIGEMMA, checkpoint_filename
    )
    PUBLISHER_MODEL_NAME = f"publishers/google/models/paligemma@{model_name}"
else:
    model_name = f"{model_name_prefix}-{resolution}-{precision_type}-custom"
    checkpoint_path = custom_paligemma_model_uri

# @markdown If you want to use other accelerator types not listed below, then check other Vertex AI prediction supported accelerators and regions at https://cloud.google.com/vertex-ai/docs/predictions/configure-compute. You may need to manually set the `machine_type`, `accelerator_type`, and `accelerator_count` in the code by clicking `Show code` first.
# @markdown Select the accelerator type to use to deploy the model:
accelerator_type = "NVIDIA_L4"  # @param ["NVIDIA_L4", "NVIDIA_TESLA_V100"]
if accelerator_type == "NVIDIA_L4":
    machine_type = "g2-standard-16"
    accelerator_count = 1
elif accelerator_type == "NVIDIA_TESLA_V100":
    if resolution == 896 and precision_type == "float32":
        raise ValueError(
            "NVIDIA_TESLA_V100 is not sufficient. Multi-gpu is not supported for PaLIGemma."
        )
    else:
        machine_type = "n1-highmem-8"
        accelerator_count = 1
else:
    raise ValueError(
        f"Recommended machine settings not found for: {accelerator_type}. To use another another accelerator, edit this code block to pass in an appropriate `machine_type`, `accelerator_type`, and `accelerator_count` to the deploy_model function by clicking `Show Code` and then modifying the code."
    )

# @markdown Set use_dedicated_endpoint to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint). Note that [dedicated endpoint does not support VPC Service Controls](https://cloud.google.com/vertex-ai/docs/predictions/choose-endpoint-type), uncheck the box if you are using VPC-SC.
use_dedicated_endpoint = True  # @param {type:"boolean"}

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    is_for_training=False,
)

# @markdown Click "Show Code" to see more details.

In [ ]:
# @title [Option 1] Deploy with Model Garden SDK

# @markdown Kindly note that the deployment using custom_paligemma_model_uri is not supported.

# @markdown Deploy with Gen AI model-centric SDK. This section uploads the prebuilt model to Model Registry and deploys it to a Vertex AI Endpoint. It takes 15 minutes to 1 hour to finish depending on the size of the model. See [use open models with Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/open-models/use-open-models) for documentation on other use cases.
from vertexai import model_garden

model = model_garden.OpenModel(PUBLISHER_MODEL_NAME)
endpoints[LABEL] = model.deploy(
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    use_dedicated_endpoint=use_dedicated_endpoint,
    accept_eula=True,  # Accept the End User License Agreement (EULA) on the model card before deploy. Otherwise, the deployment will be forbidden.
)

endpoint = endpoints[LABEL]

In [ ]:
# @title [Option 2] Deploy with custom configs

# @markdown This section uploads the prebuilt PaliGemma model to Model Registry and deploys it to a Vertex AI Endpoint. It takes approximately 15 minutes to finish.

# The pre-built serving docker image.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/jax-paligemma-serve-gpu:20240807_0916_RC00"


def deploy_model(
    model_name: str,
    checkpoint_path: str,
    machine_type: str = "g2-standard-32",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    resolution: int = 224,
    use_dedicated_endpoint: bool = False,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Create a Vertex AI Endpoint and deploy the specified model to the endpoint."""
    model_name_with_time = common_util.get_job_name_with_datetime(model_name)
    endpoint = aiplatform.Endpoint.create(
        display_name=f"{model_name_with_time}-endpoint",
        dedicated_endpoint_enabled=use_dedicated_endpoint,
    )
    model = aiplatform.Model.upload(
        display_name=model_name_with_time,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[8080],
        serving_container_predict_route="/predict",
        serving_container_health_route="/health",
        serving_container_environment_variables={
            "CKPT_PATH": checkpoint_path,
            "RESOLUTION": resolution,
            "MODEL_ID": "google/" + model_name,
            "DEPLOY_SOURCE": "notebook",
        },
        model_garden_source_model_name="publishers/google/models/paligemma",
    )
    print(
        f"Deploying {model_name_with_time} on {machine_type} with {accelerator_count} {accelerator_type} GPU(s)."
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        enable_access_logging=True,
        min_replica_count=1,
        sync=True,
        system_labels={
            "NOTEBOOK_NAME": "model_garden_jax_paligemma_deployment.ipynb",
            "NOTEBOOK_ENVIRONMENT": common_util.get_deploy_source(),
        },
    )
    return model, endpoint


models[LABEL], endpoints[LABEL] = deploy_model(
    model_name=model_name,
    checkpoint_path=checkpoint_path,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    resolution=resolution,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

In [ ]:
# @title [Optional] Loading an existing Endpoint
# @markdown If you've already deployed an Endpoint, you can load it by filling in the Endpoint's ID below.
# @markdown You can view deployed Endpoints at [Vertex Online Prediction](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).
endpoint_id = ""  # @param {type: "string"}

if endpoint_id:
    endpoints[LABEL] = aiplatform.Endpoint(
        endpoint_name=endpoint_id,
        project=PROJECT_ID,
        location=REGION,
    )

### Predict

The following sections will use images from [pexels.com](https://www.pexels.com/) for demoing purposes. All the images have the following license: https://www.pexels.com/license/.

Images will be resized to a width of 1000 pixels by default since requests made to a Vertex Endpoint are limited to 1.500MB.

In [ ]:
# @title Visual Question Answering

# @markdown This section uses the deployed PaliGemma model to answer questions about a given image.

# @markdown Once deployment succeeds, you can send requests to the endpoint with images and questions.
# @markdown ![](https://images.pexels.com/photos/4012966/pexels-photo-4012966.jpeg?w=1260&h=750)
image_url = "https://images.pexels.com/photos/4012966/pexels-photo-4012966.jpeg"  # @param {type:"string"}

image = common_util.download_image(image_url)
display(image)

# @markdown You may leave question prompts empty and they will be ignored.
question_prompt_1 = "Which of laptop, book, pencil, clock, flower are in the image?"  # @param {type: "string"}
question_prompt_2 = "Do the book and the cup have the same color?"  # @param {type: "string"}
question_prompt_3 = "Is there a person in the image?"  # @param {type: "string"}
question_prompt_4 = "How many laptop are in the image?"  # @param {type: "string"}
question_prompt_5 = "桌子是什么颜色的?"  # @param {type: "string"}

# @markdown The question prompt can be non-English languages.
questions_list = [
    question_prompt_1,
    question_prompt_2,
    question_prompt_3,
    question_prompt_4,
    question_prompt_5,
]
questions = [question for question in questions_list if question]

answers = common_util.vqa_predict(
    endpoints["endpoint"],
    questions,
    image,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

for question, answer in zip(questions, answers):
    print(f"Question: {question}")
    print(f"Answer: {answer}")
# @markdown Click "Show Code" to see more details.

In [ ]:
# @title Image Captioning
# @markdown This section uses the deployed PaliGemma model to caption and describe an image in a chosen language.

caption_prompt = True

# @markdown <img src="https://storage.googleapis.com/longcap100/91.jpeg" width="400" >

image_url = "https://storage.googleapis.com/longcap100/91.jpeg"  # @param {type:"string"}

language_code = "en"  # @param {type: "string"}

image = common_util.download_image(image_url)
display(image)

# Make a prediction.
image_base64 = common_util.image_to_base64(image)

caption = common_util.caption_predict(
    endpoints["endpoint"],
    language_code,
    image,
    caption_prompt,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

print("Caption: ", caption)
# @markdown Click "Show Code" to see more details.

In [ ]:
# @title OCR
# @markdown This section uses the deployed PaliGemma model to extract text from an image, starting from the top left.
ocr_prompt = "ocr"

# @markdown ![](https://images.pexels.com/photos/8919535/pexels-photo-8919535.jpeg?auto=compress&cs=tinysrgb&w=630&h=375&dpr=2)
image_url = "https://images.pexels.com/photos/8919535/pexels-photo-8919535.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2"  # @param {type:"string"}

image = common_util.download_image(image_url)
display(image)
text_found = common_util.ocr_predict(
    endpoints["endpoint"],
    ocr_prompt,
    image,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

print(f"Text found: {text_found}")
# @markdown Click "Show Code" to see more details.

In [ ]:
# @title Object Detection
# @markdown This section uses the deployed PaliGemma model to output bounding boxes for specified object image in a given image.
# @markdown The text output will be parsed into bounding boxes and overlaid on the original image.

# @markdown Specify what object to detect. To specify multiple objects, enter them as a semicolon separated list as shown below.
objects = "plant ; pineapple ; glasses"  # @param {type:"string"}
detect_promt = f"detect {objects}"


def parse_detections(txt):
    """Parses bounding boxes from a detection string."""
    bboxes = []
    for loc_text in txt.split(" ; "):
        m = re.match(
            r"<loc(?P<y0>\d\d\d\d)><loc(?P<x0>\d\d\d\d)><loc(?P<y1>\d\d\d\d)><loc(?P<x1>\d\d\d\d)>.*",
            loc_text,
        )
        if m is not None:
            d = m.groupdict()
        else:
            raise ValueError(f"{txt} is not a value detection string.")

        def fmt_box(x):
            return float(x) / 1024.0

        box = np.array(
            [fmt_box(d["y0"]), fmt_box(d["x0"]), fmt_box(d["y1"]), fmt_box(d["x1"])]
        )
        bboxes.append(box)
    return bboxes


def plot_bounding_boxes(im: Image.Image, bboxes: Sequence[np.ndarray]) -> Image.Image:
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(im, zorder=-1)
    ax.set_xlim(*ax.get_xlim())
    ax.set_ylim(*ax.get_ylim())

    for y0, x0, y1, x1 in bboxes:
        box = np.array([y0, x0, y1, x1])
        w, h = im.size
        y1, x1, y2, x2 = box * [h, w, h, w]
        ax.add_patch(
            mpl.patches.Rectangle(
                (x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor="r", facecolor="none"
            )
        )
    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    return Image.open(buf)


# @markdown ![](https://images.pexels.com/photos/1006293/pexels-photo-1006293.jpeg?auto=compress&cs=tinysrgb&w=630&h=375&dpr=2)
image_url = "https://images.pexels.com/photos/1006293/pexels-photo-1006293.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2"  # @param {type:"string"}

image = common_util.download_image(image_url)
display(image)

# Make a prediction.
detection_response = common_util.detect_predict(
    endpoints["endpoint"],
    detect_promt,
    image,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

print("Output: ", detection_response)


bboxes = parse_detections(detection_response)
plot_bounding_boxes(image, bboxes)
# @markdown Click "Show Code" to see more details.

## Creating a webpage playground with Gradio

In [ ]:
# @title How to use
# @markdown This is a playground similar to the popular [Stable Diffusion WebUI](https://github.com/AUTOMATIC1111/stable-diffusion-webui).

# @markdown **Prerequisites**
# @markdown -  Before you can upload an image to make a prediction, you need to select a Vertex prediction endpoint serving PaliGemma
# @markdown from the endpoint dropdown list that has been deployed in the current project and region.
# @markdown -  If no models have been deployed, you can create a new Vertex prediction
# @markdown endpoint by clicking "Deploy to Vertex" in the playground or running the `Deploy` cell above.
# @markdown   * New model deployment takes approximately 15 minutes. You can check the progress at [Vertex Online Prediction](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).

# @markdown **How to use**

# @markdown Just run this cell and a link to the playground formatted as `https://####.gradio.live` will be outputted.
# @markdown This link will take you to the playground in a separate browser tab.


class Task(enum.Enum):
    VQA = "Visual Question Answering"
    CAPTION = "Image Captioning"
    OCR = "OCR"
    DETECT = "Object Detection"


def list_paligemma_endpoints() -> list[str]:
    """Returns all valid prediction endpoints for in the project and region."""
    # Gets all the valid endpoints in the project and region.
    endpoints = aiplatform.Endpoint.list(order_by="create_time desc")
    # Filters out the endpoints which do not have a deployed model, and the endpoint is for image generation
    endpoints = list(
        filter(
            lambda endpoint: endpoint.traffic_split
            and "pali" in endpoint.display_name.lower(),
            endpoints,
        )
    )

    endpoint_names = list(
        map(
            lambda endpoint: f"{endpoint.name} - {endpoint.display_name[:40]}",
            endpoints,
        )
    )

    if not endpoint_names:
        gr.Warning("No prediction endpoints were found. Create an Endpoint first.")

    return endpoint_names


def get_endpoint(endpoint_name: str) -> aiplatform.Endpoint:
    """Returns a Vertex endpoint for the given endpoint_name."""
    endpoint_id = endpoint_name.split(" - ")[0]
    endpoint = aiplatform.Endpoint(
        f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_id}"
    )
    return endpoint


def select_interface(interface_name: str):
    if interface_name == Task.VQA.value:
        return {
            text_input_box: gr.update(label="Question", value=None, visible=True),
            language_code_box: gr.update(visible=False),
            submit_button: gr.update(value="Answer"),
            text_output: gr.update(value=None),
            image_output: gr.update(value=None, visible=False),
        }
    elif interface_name == Task.CAPTION.value:
        return {
            text_input_box: gr.update(value=None, visible=False),
            language_code_box: gr.update(visible=True),
            submit_button: gr.update(value="Caption"),
            text_output: gr.update(value=None),
            image_output: gr.update(value=None, visible=False),
        }
    elif interface_name == Task.OCR.value:
        return {
            text_input_box: gr.update(value=None, visible=False),
            language_code_box: gr.update(visible=False),
            submit_button: gr.update(value="Extract text"),
            text_output: gr.update(value=None),
            image_output: gr.update(value=None, visible=False),
        }
    elif interface_name == Task.DETECT.value:
        return {
            text_input_box: gr.update(label="Object(s)", value=None, visible=True),
            language_code_box: gr.update(visible=False),
            submit_button: gr.update(value="Detect"),
            text_output: gr.update(value=None),
            image_output: gr.update(value=None, visible=True),
        }
    else:
        raise gr.Error(f"Invalid interface name: {interface_name}")


def deploy_model_handler(model_choice: str) -> None:
    gr.Info("Starting model deployment.")
    model_name = model_choice.replace("-pt-", "-")
    checkpoint_filename = pretrained_filename_lookup[model_name]
    _, _, resolution, _ = model_choice.split("-")
    resolution = int(resolution)
    model, endpoint = deploy_model(
        model_name=model_choice,
        checkpoint_path=os.path.join(
            VERTEX_AI_MODEL_GARDEN_PALIGEMMA, checkpoint_filename
        ),
        machine_type="g2-standard-16",
        accelerator_type="NVIDIA_L4",
        accelerator_count=1,
        resolution=resolution,
    )
    gr.Info(f"Deploying model ID: {model.name}, endpoint ID: {endpoint.name}")


def predict_handler(
    interface_name: str,
    endpoint_name: str,
    image: Image.Image,
    prompt: str,
    language_code: str,
) -> Tuple[str, Image.Image]:
    if not endpoint_name:
        raise gr.Error("Select (or deploy) a model first!")
    if not image:
        raise gr.Error("You must upload an image!")
    endpoint = get_endpoint(endpoint_name)
    if interface_name == Task.VQA.value:
        return common_util.vqa_predict(endpoint, [prompt], image)[0], None
    elif interface_name == Task.CAPTION.value:
        return common_util.caption_predict(endpoint, language_code, image, True), None
    elif interface_name == Task.OCR.value:
        return common_util.ocr_predict(endpoint, ocr_prompt, image), None
    elif interface_name == Task.DETECT.value:
        text_output = common_util.detect_predict(endpoint, f"detect {prompt}", image)
        bboxes = parse_detections(text_output)
        return text_output, plot_bounding_boxes(image, bboxes)
    else:
        raise gr.Error("Select an interface first!")


tip_text = r"""
<b> Tips: </b>
1. Select a Vertex prediction endpoint with a deployed PaLIGemma model or click `Deploy to Vertex` to deploy PaLIGemma to Vertex.
2. New model deployment takes approximately 15 minutes. You can check the progress at [Vertex Online Prediction](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).
3. After the model deployment is complete, click `Refresh Endpoints list` to view the new endpoint in the dropdown list.
"""

css = """
.gradio-container {
  width: 85% !important
}
"""
with gr.Blocks(
    css=css, theme=gr.themes.Default(primary_hue="orange", secondary_hue="blue")
) as demo:
    gr.Markdown("# Model Garden Playground for PaliGemma")
    with gr.Row(equal_height=True):
        with gr.Column(scale=3):
            gr.Markdown(tip_text)
        with gr.Column(scale=2):
            with gr.Row():
                endpoint_name = gr.Dropdown(
                    scale=7,
                    label="Select a model previously deployed on Vertex",
                    choices=list_paligemma_endpoints(),
                    value=None,
                )
                refresh_button = gr.Button(
                    "Refresh Endpoints list",
                    scale=1,
                    variant="primary",
                    min_width=10,
                )
            with gr.Row():
                selected_model = gr.Dropdown(
                    scale=7,
                    label="Deploy a new model to Vertex",
                    choices=[
                        "paligemma-mix-224-float32",
                        "paligemma-mix-224-float16",
                        "paligemma-mix-224-bfloat16",
                        "paligemma-mix-448-float32",
                        "paligemma-mix-448-float16",
                        "paligemma-mix-448-bfloat16",
                        "paligemma-pt-224-float32",
                        "paligemma-pt-224-float16",
                        "paligemma-pt-224-bfloat16",
                        "paligemma-pt-448-float32",
                        "paligemma-pt-448-float16",
                        "paligemma-pt-448-bfloat16",
                        "paligemma-pt-896-float32",
                        "paligemma-pt-896-float16",
                        "paligemma-pt-896-bfloat16",
                    ],
                    value=None,
                )
                deploy_model_button = gr.Button(
                    "Deploy a new model",
                    scale=1,
                    variant="primary",
                    min_width=10,
                )
    with gr.Row(equal_height=True):
        with gr.Column(scale=1):
            image_input = gr.Image(
                show_label=True,
                type="pil",
                label="Upload",
                visible=True,
                height=400,
            )
            with gr.Group():
                with gr.Tab("Task"):
                    interfaces_box = gr.Radio(
                        show_label=False,
                        choices=[
                            Task.VQA.value,
                            Task.CAPTION.value,
                            Task.OCR.value,
                            Task.DETECT.value,
                        ],
                        value=Task.VQA.value,
                    )
                text_input_box = gr.Textbox(label="Question", lines=1)
                language_code_box = gr.Textbox(
                    value="en", label="Language code", lines=1, visible=False
                )
                submit_button = gr.Button("Answer", variant="primary")
        with gr.Column(scale=1):
            image_output = gr.Image(label="Image response:", visible=False)
            text_output = gr.Textbox(label="Text response:")

    refresh_button.click(
        fn=lambda: gr.update(choices=list_paligemma_endpoints()),
        outputs=[endpoint_name],
    )
    deploy_model_button.click(
        deploy_model_handler,
        inputs=[selected_model],
        outputs=[],
    )
    interfaces_box.change(
        fn=select_interface,
        inputs=interfaces_box,
        outputs=[
            text_input_box,
            language_code_box,
            submit_button,
            text_output,
            image_output,
        ],
    )
    submit_button.click(
        fn=predict_handler,
        inputs=[
            interfaces_box,
            endpoint_name,
            image_input,
            text_input_box,
            language_code_box,
        ],
        outputs=[text_output, image_output],
    )
show_debug_logs = True  # @param {type: "boolean"}
demo.queue()
demo.launch(
    share=True, inline=False, inbrowser=True, debug=show_debug_logs, show_error=True
)

# @markdown Click "Show Code" to see more details.

## Clean up resources

In [ ]:
# @title Delete the models and endpoints

# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()